In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
from keras.datasets import cifar10

(x_train_orig, y_train_orig), (x_test_orig, y_test_orig) = cifar10.load_data()

Using TensorFlow backend.


In [3]:
print(x_train_orig.shape, y_train_orig.shape)
print(x_test_orig.shape, y_test_orig.shape)

(50000, 32, 32, 3) (50000, 1)
(10000, 32, 32, 3) (10000, 1)


In [4]:
def get_one_hot(targets, nb_classes):
    res = np.eye(nb_classes)[np.array(targets).reshape(-1)]
#     return res.reshape(list(targets.shape)+[nb_classes])
    return res

In [17]:
x_train = x_train_orig[:10000,] / 255
y_train = get_one_hot(y_train_orig[:10000,:], 10)
y_test = y_test_orig / 255
y_test = get_one_hot(y_test_orig, 10)

In [18]:
print(y_train.shape, y_test.shape)

(10000, 10) (10000, 10)


In [19]:
def creat_place_holder(n_H, n_W, n_C, n_y):
    X = tf.placeholder(tf.float32, shape=(None, n_H, n_W, n_C), name="X")
    y = tf.placeholder(tf.float32, shape=(None, n_y), name="y")
    
    return X, y

In [20]:
def init_param():
#     [filter_height, filter_width, in_channels, out_channels]
    
    W1= tf.get_variable(shape=(5,5,3,8), initializer=tf.contrib.layers.xavier_initializer(), name="W1")
    W2= tf.get_variable(shape=(7,7,8,16), initializer=tf.contrib.layers.xavier_initializer(), name="W2")
    
    param = {
        "W1" : W1,
        "W2" : W2
    }
    
    return param

In [21]:
def forward_propagation(X, param):
    """
    CONV2D -> RELU -> MAXPOOL -> CONV2D -> RELU -> MAXPOOL -> FLATTEN -> FULLYCONNECTED
    """
    W1 = param["W1"]
    W2 = param["W2"]
    
    Z1 = tf.nn.conv2d(input=X, filter=W1, strides=(1,1,1,1), padding="SAME", name="conv1")
    A1 = tf.nn.relu(Z1, name="relu1")
    P1 = tf.nn.max_pool(A1, ksize=[1,2,2,1], strides=[1,2,2,1], padding="VALID", name="maxpool1")
    
    Z2 = tf.nn.conv2d(input=P1, filter=W2, strides=(1,1,1,1), padding="SAME", name="conv2")
    A2 = tf.nn.relu(Z2, name="relu2")
    P2 = tf.nn.max_pool(A2, ksize=[1,2,2,1], strides=[1,2,2,1], padding="VALID", name="maxpool2")
    
    F1 = tf.layers.flatten(P2)
    Z3 = tf.contrib.layers.fully_connected(inputs=F1, num_outputs=10, activation_fn=None)
    
    cache = {
        "Z1" : Z1,
        "Z2" : Z2,
        "Z3" : Z3,
        "A1" : A1,
        "A2" : A2,
        "P1" : P1,
        "P2" : P2,
        "F1" : F1
    }
    
    return cache

In [22]:
def compute_cost(Z3, y):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=Z3, labels=y))
    return cost

In [23]:
def model(X_train, y_train, learning_rate = 0.0001, num_epochs=100):
    
    tf.reset_default_graph()
    
    (m, n_H, n_W, n_C) = X_train.shape
    (m, n_y) = y_train.shape
    
    X, y = creat_place_holder(n_H, n_W, n_C, n_y)
    param = init_param()
    cache = forward_propagation(X, param)
    
    Z3 = cache["Z3"]
    
    cost = compute_cost(Z3, y)
    
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

    init = tf.global_variables_initializer()
     
    with tf.Session() as sess:
        
        sess.run(init)
        
        costs = []
        
        for epoch in range(num_epochs):
            
            cost_epoch, _ = sess.run([cost, optimizer], feed_dict={X:X_train, y:y_train})
            
            if(epoch%5 == 0):
                costs.append(cost_epoch)
                print(epoch, cost_epoch)

In [1]:
model(x_train, y_train)

NameError: name 'model' is not defined